<a href="https://colab.research.google.com/github/apeksha104/-DCode---Simplifying-Privacy-Agreements/blob/main/Semantic_Comparator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim==4.1.2


In [ ]:
pip install --upgrade gensim


     |████████████████████████████████| 24.1 MB 2.8 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
from re import sub
import gensim
from gensim.utils import simple_preprocess
import numpy as np
from gensim import interfaces, utils, matutils
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity


#Stop words are a set of commonly used words in a language. Examples of stop words in English are “a”, “the”, “is”, “are” and etc. 
#Stop words are commonly used in Text Mining and Natural Language Processing (NLP) to eliminate words that are so commonly used that they carry very little useful information.


stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
from textblob import TextBlob

#Answers from Adobe Website
q1 = ['Adobe websites use cookies and similar technologies provided by our own company and other companies that allow us to gather additional information to measure and improve the effectiveness of our advertising.','We will also share your personal information with companies that help us to run our business by processing personal information on behalf of Adobe.','Where you give appropriate permissions, we will receive information about you from your social networking account, such as name, location and basic demographic information.','There are several places within Adobes websites and apps that allow you to post comments, upload pictures or submit content for others to see.']

#Answers from the internet
d1 = ['Our websites use cookies and other technologies by our company for better advertising and effectiveness.','Personal information will be shared with other companies to run our business better by processing data.','We will receive information from your social accounts when appropriate permissions are given by you.','Your content would be visible to others, such as the comments that you post and the pictures that you upload on the Adobe websites.']


def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

# Preprocess the documents, including the query string
scores = []
for f, b in zip(q1, d1):
    documents = ['']
    query_string = f
    documents.append(b)
    corpus = [preprocess(document) for document in documents]
    query = preprocess(query_string)
# Load the model: this is a big file, can take a while to download and open
    glove = api.load("glove-wiki-gigaword-50")    
    similarity_index = WordEmbeddingSimilarityIndex(glove)

# Build the term dictionary, TF-idf model
    dictionary = Dictionary(corpus+[query])
    tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
    similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

    query_tf = tfidf[dictionary.doc2bow(query)]

    index = SoftCosineSimilarity(
                tfidf[[dictionary.doc2bow(document) for document in corpus]],
                similarity_matrix)

    doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
    sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
    for idx in sorted_indexes:
        print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')
        scores.append(doc_similarity_scores[idx])

    documents.clear()
 

    
def Average(lst):
    return sum(lst) / 4
    
average = Average(scores)
print("Average comparator score =", round(average, 2))


    
    

[==================================================] 100.0% 66.0/66.0MB downloaded


100%|██████████| 19/19 [00:00<00:00, 42.58it/s]
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


1 	 0.567 	 Our websites use cookies and other technologies by our company for better advertising and effectiveness.
0 	 0.000 	 


100%|██████████| 15/15 [00:00<00:00, 42.02it/s]
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


1 	 0.569 	 Personal information will be shared with other companies to run our business better by processing data.
0 	 0.000 	 


100%|██████████| 14/14 [00:00<00:00, 42.04it/s]
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


1 	 0.742 	 We will receive information from your social accounts when appropriate permissions are given by you.
0 	 0.000 	 


100%|██████████| 18/18 [00:00<00:00, 43.81it/s]

1 	 0.563 	 Your content would be visible to others, such as the comments that you post and the pictures that you upload on the Adobe websites.
0 	 0.000 	 
Average comparator score = 0.61



/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


In [ ]:
print(similarity_matrix)

In [ ]:
print(dictionary)

Dictionary(18 unique tokens: ['adobe', 'comments', 'content', 'others', 'pictures']...)


In [ ]:
print(doc_similarity_scores)

[0.         0.56285924]


In [ ]:
print(gensim.__version__)

4.1.2
